In [3]:
import os
import matplotlib.pyplot as plt
import numpy as np

import cv2
from math import sin, radians

from keras_ssd512 import ssd_512

from architectures import mpatacchiola_generic
from head_detector_utils import get_head_bboxes, get_cropped_pics
from pose_estimator_utils import get_pose

# Paths

detector_file = 'head-detector.h5'
estimator_file = 'pose-estimator.h5'

models_path = 'models/'

detector_path = models_path + detector_file
estimator_path = models_path + estimator_file

# Detector parameters.

in_size_detector = 512
confidence_threshold = 0.2

# Estimator parameters.

in_size_estimator = 64
num_conv_blocks = 6
num_filters_start = 32
num_dense_layers = 1
dense_layer_size = 512

# Normalization parameters.

mean = 0.408808
std = 0.237583

t_mean = -0.041212
t_std = 0.323931

p_mean = -0.000276
p_std = 0.540958

# Models.

head_detector = ssd_512(image_size=(in_size_detector, in_size_detector, 3), n_classes=1, min_scale=0.1, max_scale=1, mode='inference')
head_detector.load_weights(detector_path)

pose_estimator = mpatacchiola_generic(in_size_estimator, num_conv_blocks, num_filters_start, num_dense_layers, dense_layer_size)
pose_estimator.load_weights(estimator_path)





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [14]:
def get_angles(example_path, num_people, result_path=None):
    tilts = [list() for i in range(num_people)]
    pans = [list() for i in range(num_people)]

    cap = cv2.VideoCapture(example_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    if result_path:
        out = cv2.VideoWriter(result_path, cv2.VideoWriter_fourcc(*'mp4v'), 
                              cap.get(cv2.CAP_PROP_FPS), (frame_width, frame_height))
    
    while cap.isOpened():
        ret, img = cap.read()

        if not ret:
            print('Could not capture a valid frame from video source, check your cam/video value...')
            break

        # Get bounding boxes for every detected head in the picture.
        bboxes = get_head_bboxes(img, head_detector, confidence_threshold)

        # Get cropped pics for every valid bounding box.
        gray_pic = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        heads = get_cropped_pics(gray_pic, bboxes, in_size_estimator, 0, cropping='small')
        # For each cropped picture:
        for i in range(len(heads)):
            

            # If it is a valid picture:
            if heads[i].shape == (in_size_estimator, in_size_estimator):

                # Get pose values.
                tilt, pan = get_pose(heads[i], pose_estimator, img_norm = [mean, std], tilt_norm = [t_mean, t_std],
                                     pan_norm = [p_mean, p_std], rescale=90.0)

                tilts[i].append(tilt)
                pans[i].append(pan)
                    
                
                 # Get minimum and maximum values for both axes of the bounding box.
                xmin, ymin, xmax, ymax = bboxes[i]

                # Draw detection in the original picture..

                rect = cv2.rectangle(img, (xmax, ymin), (xmin, ymax), (0, 255, 0), 2, lineType=cv2.LINE_AA)
                cv2.putText(rect, 'TILT: ' + str(round(tilt, 2)) + ' PAN: ' + str(round(pan, 2)), (xmin, ymin - 10), cv2.FONT_HERSHEY_DUPLEX, 0.5, (0, 0, 255), 1)

                # Draw arrow from the center of the picture in the direction of the pose in the original picture.

                centerx = int((xmin + xmax) / 2)
                centery = int((ymin + ymax) / 2)
                center = (centerx, centery)

                max_arrow_len = (xmax - xmin + 1) / 2

                offset_x = -1 * int(sin(radians(pan)) * max_arrow_len)
                offset_y = -1 * int(sin(radians(tilt)) * max_arrow_len)

                end = (centerx + offset_x, centery + offset_y)
                cv2.arrowedLine(img, center, end, (0, 0, 255), 2, line_type=cv2.LINE_AA)
                
        if result_path:
            out.write(img)
        
    cap.release()
    if result_path:
        out.release()
        
    return np.array(tilts), np.array(pans)

In [7]:
examples_path = os.path.join("..", "test_videos", "examples")
results_path = os.path.join("..", "test_videos", "results")
results_arrays_path = os.path.join("..", "test_videos", "results_arrays")

[[280, 171, 471, 453], [907, 281, 995, 418]]


In [11]:
for i in range(1, 8):
    path = os.path.join(examples_path, "1", f"{i}.mp4")
    get_angles(path)

[[868, 178, 1096, 467], [296, 285, 390, 426]]
[[859, 184, 1089, 465], [286, 305, 366, 431]]
[[869, 193, 1089, 462], [299, 298, 374, 434]]
[[885, 187, 1106, 457], [295, 294, 376, 419]]
[[837, 191, 1059, 458], [302, 305, 379, 432]]
[[861, 180, 1091, 455], [297, 301, 376, 427]]
[[862, 192, 1089, 459], [297, 305, 379, 439]]


In [12]:
for i in range(1, 5):
    path = os.path.join(examples_path, "2", f"{i}.mp4")
    get_angles(path)

[[280, 171, 471, 453], [907, 281, 995, 418]]
[[296, 165, 483, 435], [914, 280, 1001, 411]]
[[321, 174, 492, 408], [918, 280, 1007, 413]]
[[315, 165, 502, 436], [903, 284, 993, 407]]


In [70]:
for (root,dirs,files) in os.walk(examples_path, topdown=True):
    if root[-1] not in ('1', '2'):    
        continue
    for file in files:
        video_path = os.path.join(root, file)
        folder = video_path.split('\\')[-2]
        file = video_path.split('\\')[-1][:-4]
        result_path = os.path.join(results_path, folder, file + ".mp4")
        results_array_path = os.path.join(results_arrays_path, folder)
        print('=======================================================')
        print(video_path)
        print(result_path)
        print(results_array_path)
        print('=======================================================')
        (tilts_1, pans_1), (tilts_2, pans_2) = get_angles(video_path, result_path)
        
        np.save(f"{results_array_path}\\{file}_tilts_1.npy", tilts_1)
        np.save(f"{results_array_path}\\{file}_tilts_2.npy", tilts_2)
        np.save(f"{results_array_path}\\{file}_pans_1.npy", pans_1)
        np.save(f"{results_array_path}\\{file}_pans_2.npy", pans_2)

..\test_videos\examples\1\1.mp4
..\test_videos\results\1\1.mp4
..\test_videos\results_arrays\1
Could not capture a valid frame from video source, check your cam/video value...
..\test_videos\examples\1\2.mp4
..\test_videos\results\1\2.mp4
..\test_videos\results_arrays\1
Could not capture a valid frame from video source, check your cam/video value...
..\test_videos\examples\1\3.mp4
..\test_videos\results\1\3.mp4
..\test_videos\results_arrays\1
Could not capture a valid frame from video source, check your cam/video value...
..\test_videos\examples\1\4.mp4
..\test_videos\results\1\4.mp4
..\test_videos\results_arrays\1
Could not capture a valid frame from video source, check your cam/video value...
..\test_videos\examples\1\5.mp4
..\test_videos\results\1\5.mp4
..\test_videos\results_arrays\1
Could not capture a valid frame from video source, check your cam/video value...
..\test_videos\examples\1\6.mp4
..\test_videos\results\1\6.mp4
..\test_videos\results_arrays\1
Could not capture a valid

## Akshita Videos

In [15]:
video1 = os.path.join('..', 'test_videos', 'Test1.mp4')
video2 = os.path.join('..', 'test_videos', 'Test2.mp4')

res_video1 = os.path.join('..', 'test_videos', 'results', 'Test1.mp4')
res_video2 = os.path.join('..', 'test_videos', 'results', 'Test2.mp4')

res_arrays_video1 = os.path.join('..', 'test_videos', 'results_arrays', 'Test1')
res_arrays_video2 = os.path.join('..', 'test_videos', 'results_arrays', 'Test2')

tilts, pans = get_angles(video1, 1, res_video1)
np.save(f"{res_arrays_video1}_tilts.npy", tilts)
np.save(f"{res_arrays_video1}_pans.npy", pans)

tilts, pans = get_angles(video2, 1, res_video2)
np.save(f"{res_arrays_video2}_tilts.npy", tilts)
np.save(f"{res_arrays_video2}_pans.npy", pans)

Could not capture a valid frame from video source, check your cam/video value...
Could not capture a valid frame from video source, check your cam/video value...
